# Load Dataset

In [1]:
from tqdm.auto import tqdm

In [2]:
from datasets import load_dataset

df = load_dataset("ai4bharat/IndicHeadlineGeneration", "ml")

In [3]:
df = df["test"].select(range(500))
print(df[:1])

{'id': ['1'], 'input': ['ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മ

In [4]:
texts = df['input']
references = df['target']

# Compute Metrics

In [5]:
import numpy as np
import evaluate
import pyiwn

# Load evaluation metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

# Initialize IndoWordNet for Malayalam
iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.MALAYALAM)

def expand_synonyms(word):
    """
    Expand a single word into its synonym set using IndoWordNet.
    If no synsets found, return the word itself.
    """
    try:
        synsets = iwn.synsets(word)
    except KeyError:
        return {word}

    if not synsets:
        return {word}

    synonyms = set()
    for syn in synsets:
        synonyms.update(syn.lemma_names())
    return synonyms if synonyms else {word}

def sentence_to_synonym_sets(sentence):
    """
    Convert a sentence into a list of synonym sets per word.
    """
    return [expand_synonyms(w) for w in sentence.split()]

def synonym_overlap(pred, ref):
    """
    Computes the proportion of words in the predicted headline 
    that have synonym matches in the reference headline.
    """
    pred_sets = sentence_to_synonym_sets(pred)
    ref_sets = sentence_to_synonym_sets(ref)

    # Precision: how many predicted words match reference
    match_pred = sum(1 for ps in pred_sets if any(ps & rs for rs in ref_sets))
    precision = match_pred / len(pred_sets) if pred_sets else 0.0

    # Recall: how many reference words are found in prediction
    match_ref = sum(1 for rs in ref_sets if any(rs & ps for ps in pred_sets))
    recall = match_ref / len(ref_sets) if ref_sets else 0.0

    # F1 Score
    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * precision * recall / (precision + recall)

    return round(f1, 4)

2025-07-01 13:43:09.603300: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-01 13:43:09.619415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751373789.638158  956381 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751373789.643944  956381 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751373789.658577  956381 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [6]:
import evaluate
import numpy as np

# Load metrics once
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

def compute_metrics(predictions, references, rouge_stemmer=True):
    """
    Computes ROUGE, BLEU, and BERTScore for given predictions and references.
    
    Args:
        predictions (list): List of predicted texts.
        references (list): List of reference texts.
        rouge_stemmer (bool): Whether to use stemming for ROUGE.
    
    Returns:
        dict: ROUGE, BLEU, and BERTScore metrics rounded to four decimal places.
    """
    
    # Compute ROUGE scores
    rouge_result = rouge.compute(predictions=predictions, references=references, use_stemmer=rouge_stemmer)
    
    # Compute BLEU scores
    bleu_result = bleu.compute(
        predictions=predictions,
        references=[[r] for r in references]
    )

    # Compute BERTScore (for Malayalam use lang="ml")
    bert_result = bertscore.compute(
        predictions=predictions,
        references=references,
        lang="ml"  # Malayalam
    )

     # Synonym-aware Rouge
    SynROUGE = round(np.mean([synonym_overlap(p, r) for p, r in zip(predictions, references)]), 4)

    return {
        "ROUGE-1": round(rouge_result['rouge1'], 4),
        "ROUGE-2": round(rouge_result['rouge2'], 4),
        "ROUGE-L": round(rouge_result['rougeL'], 4),
        "BLEU": round(bleu_result["bleu"], 4),
        "BERTScore-P": round(np.mean(bert_result["precision"]), 4),
        "BERTScore-R": round(np.mean(bert_result["recall"]), 4),
        "BERTScore-F1": round(np.mean(bert_result["f1"]), 4),
        "Synonym-Aware-ROUGE(SynROUGE)": SynROUGE
    }


# Print Results

In [7]:
def print_results(predictions, num_results = 5):
    for i in range(num_results):
        print(f"Input Text:\n{texts[i]}\n")
        print(f"Generated Summary:\n{predictions[i]}\n")
        print(f"Reference Summary:\n{references[i]}\n")
        print("="*80)

# IndicBart

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "ai4bharat/IndicBART"  # Malayalam IndicBART

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, use_fast=False, keep_accents=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(64014, 1024, padding_idx=0)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(64014, 1024, padding_idx=0)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True

In [9]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")


In [10]:
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator


def malayalam_to_devanagari(input_text):
    # Check if input is a string or a list of strings
    if isinstance(input_text, str):
        return UnicodeIndicTransliterator.transliterate(input_text, 'ml', 'hi') + ' </s> <2hi>'
    elif isinstance(input_text, list):
        # If it's a list, apply the transformation to each string
        return [UnicodeIndicTransliterator.transliterate(text, 'ml', 'hi') + ' </s> <2hi>' for text in input_text]
    else:
        raise ValueError("Input must be a string or a list of strings.")

def devanagari_to_malayalam(input_text):
    # Check if input is a string or a list of strings
    if isinstance(input_text, str):
        return UnicodeIndicTransliterator.transliterate(input_text, 'hi', 'ml')
    elif isinstance(input_text, list):
        # If it's a list, apply the transformation to each string
        return [UnicodeIndicTransliterator.transliterate(text, 'hi', 'ml') for text in input_text]
    else:
        raise ValueError("Input must be a string or a list of strings.")

In [11]:
batch_size = 8  # smaller batch size to show progress in chunks
predictions = []

for i in tqdm(range(0, len(texts), batch_size), desc="Generating summaries"):
    batch_texts = texts[i:i+batch_size]
    batch_texts = malayalam_to_devanagari(batch_texts)
    # inputs = tokenizer(batch_texts, max_length=512, truncation=True, padding=True, return_tensors='pt')
    # inputs = tokenizer("मैं [MASK] हूँ </s> <2hi>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids
    inputs = tokenizer(batch_texts, add_special_tokens=False, padding=True, return_tensors='pt')

    # summary_ids = model.generate(
    #     inputs['input_ids'], attention_mask=inputs['attention_mask'],
    #     max_length=128, num_beams=4, early_stopping=True
    # )

    summary_ids = model.generate(
        inputs['input_ids'], use_cache=True, num_beams=4, max_length=128,
        early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id,
        eos_token_id=eos_id,
        decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>")
        )

    batch_preds = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    predictions.extend(batch_preds)

predictions = devanagari_to_malayalam(predictions)

Generating summaries:   0%|          | 0/63 [00:00<?, ?it/s]

In [12]:
compute_metrics(predictions, references)

2025-07-01:11:21:09,98 INFO     [rouge_scorer.py:83] Using default tokenizer.


{'ROUGE-1': np.float64(0.0505),
 'ROUGE-2': np.float64(0.0108),
 'ROUGE-L': np.float64(0.0507),
 'BLEU': 0.0222,
 'BERTScore-P': np.float64(0.6125),
 'BERTScore-R': np.float64(0.7737),
 'BERTScore-F1': np.float64(0.6832),
 'Synonym-Aware-ROUGE(SynROUGE)': np.float64(0.1774)}

In [13]:
print_results(predictions)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# Qwen

In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
model_id = "Qwen/Qwen2-1.5B-Instruct"


device = "cuda:1" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", trust_remote_code=True).to(device)
model.eval()
torch.cuda.empty_cache()

In [32]:
predictions=[]

for text in tqdm(texts, desc="Generating summaries"):
  try:
    prompt = f"""
      Generate a short and clear news headline in Malayalam based on the following text.

      Text to generate headline for: {text}

      The headline should capture the main idea of the text without copying it word-for-word.
      Return the headline in complete Malayalam.
      You are not to return the thought process or any non Malayalam texts.
      """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    with torch.no_grad():
      generated_ids = model.generate(
          model_inputs.input_ids,
          max_new_tokens=512
      )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # if "Summary:" in response:
    #   summary_clean = response.split("Summary:")[-1].strip()
    # else:
    #   summary_clean = response.strip()

    predictions.append(response)

    torch.cuda.empty_cache()

  except torch.cuda.OutOfMemoryError:
    print("⚠️ CUDA OOM – Skipping this input.")
    torch.cuda.empty_cache()
    predictions.append("[OOM ERROR – Skipped]")

Generating summaries:   0%|          | 0/500 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [33]:
compute_metrics(predictions, references)


2025-07-01:12:13:44,241 INFO     [rouge_scorer.py:83] Using default tokenizer.


{'ROUGE-1': np.float64(0.0239),
 'ROUGE-2': np.float64(0.002),
 'ROUGE-L': np.float64(0.0237),
 'BLEU': 0.0275,
 'BERTScore-P': np.float64(0.6887),
 'BERTScore-R': np.float64(0.7095),
 'BERTScore-F1': np.float64(0.698),
 'Synonym-Aware-ROUGE(SynROUGE)': np.float64(0.1392)}

In [34]:
print_results(predictions)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# Gemma



In [26]:
from huggingface_hub import login
TOKEN=None
login(token=TOKEN)

In [ ]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch

model_id = "google/gemma-3-4b-it"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = Gemma3ForConditionalGeneration.from_pretrained(model_id).to(device).eval()

processor = AutoProcessor.from_pretrained(model_id, use_fast=True)
torch.set_float32_matmul_precision('high')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 1 has a total capacity of 79.25 GiB of which 39.62 MiB is free. Process 849220 has 19.97 GiB memory in use. Process 858954 has 414.00 MiB memory in use. Process 870637 has 32.18 GiB memory in use. Process 873209 has 10.80 GiB memory in use. Process 931549 has 416.00 MiB memory in use. Including non-PyTorch memory, this process has 15.40 GiB memory in use. Of the allocated memory 14.77 GiB is allocated by PyTorch, and 231.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [ ]:
predictions = []
torch.cuda.empty_cache()

for text in tqdm(texts, desc="Generating summaries"):
  try:
    prompt = f"""
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""

    messages = [
        [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant that generates news headlines."},]
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt},]
            },
        ],
    ]

    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
      generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
      generation = generation[0][input_len:]

    # Slice off input tokens from generated tokens to get only new tokens
    # generated_only_ids = [
    #     gen_ids[len(input_ids[idx]):] for idx, gen_ids in enumerate(generated_ids)
    # ]

    decoded = processor.decode(generation, skip_special_tokens=True)
    predictions.append(decoded)

    torch.cuda.empty_cache()

  except torch.cuda.OutOfMemoryError:
    print("⚠️ CUDA OOM – Skipping this input.")
    torch.cuda.empty_cache()
    predictions.append("[OOM ERROR – Skipped]")

TypeError: 'module' object is not callable

In [29]:
import re

predictions = [re.sub(r"(?i)^\s*Headline:\s*", "", p) for p in predictions]

In [30]:
compute_metrics(predictions, references)

{'ROUGE-1': np.float64(0.0242),
 'ROUGE-2': np.float64(0.006),
 'ROUGE-L': np.float64(0.0245),
 'BLEU': 0.0205,
 'BERTScore-P': np.float64(0.7366),
 'BERTScore-R': np.float64(0.7037),
 'BERTScore-F1': np.float64(0.719)}

In [31]:
print_results(predictions)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# By-T5-Small

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Load ByT5 and Flan-T5
byt5_model = AutoModelForSeq2SeqLM.from_pretrained("google/byt5-small").to(device).eval()
byt5_tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

In [9]:
def generate_summary(text, model, tokenizer):
    prompt = f"""
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512, padding=True).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            num_beams=2,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()


In [10]:
byt5_preds = []

for text in tqdm(texts, desc="ByT5-Small"):
    generated_summary = generate_summary(text, byt5_model, byt5_tokenizer)
    byt5_preds.append(generated_summary)

ByT5-Small:   0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
import re

# Remove any English word followed by a colon and text inside parentheses
byt5_preds = [re.sub(r"(?i)^\s*\w+:\s*|\s*\([^)]*\)|:\s*", "", p) for p in byt5_preds]


In [12]:
compute_metrics(byt5_preds, references)

2025-07-01:12:54:31,421 INFO     [rouge_scorer.py:83] Using default tokenizer.


{'ROUGE-1': np.float64(0.0023),
 'ROUGE-2': np.float64(0.0),
 'ROUGE-L': np.float64(0.002),
 'BLEU': 0.0088,
 'BERTScore-P': np.float64(0.6443),
 'BERTScore-R': np.float64(0.6131),
 'BERTScore-F1': np.float64(0.6275),
 'Synonym-Aware-ROUGE(SynROUGE)': np.float64(0.0691)}

In [13]:
print_results(byt5_preds)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# **mT5**


In [14]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/umt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/umt5-base").eval()

In [15]:
import torch
torch.cuda.empty_cache()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device);

In [16]:
prompt_template = """
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""
predictions = []
batch_size = 8  # Adjust depending on GPU memory

for i in tqdm(range(0, len(texts), batch_size), desc="Generating summaries"):
    batch_texts = texts[i:i+batch_size]

    prompts = [prompt_template.format(text=text) for text in batch_texts]

    # Tokenize inputs
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
    
    ######################################################
    inputs.pop("token_type_ids", None) # used only with umt
    ######################################################

    # Generate summaries
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            early_stopping=True,
            num_beams=4,
            length_penalty=1.0,
            no_repeat_ngram_size=3,
        )

    # Decode outputs
    decoded_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    decoded_batch = [d.replace("summary:", "").strip() for d in decoded_batch]
    predictions.extend(decoded_batch)

# Now predictions has your model summaries for all texts

Generating summaries:   0%|          | 0/63 [00:00<?, ?it/s]

In [17]:
predictions = [re.sub(r"(?i)^\s*\w+:\s*|\s*\([^)]*\)|:\s*", "", p) for p in predictions]

In [21]:
compute_metrics(predictions,references)

2025-07-01:13:33:26,472 INFO     [rouge_scorer.py:83] Using default tokenizer.


{'ROUGE-1': np.float64(0.0117),
 'ROUGE-2': np.float64(0.002),
 'ROUGE-L': np.float64(0.0116),
 'BLEU': 0.0055,
 'BERTScore-P': np.float64(0.5837),
 'BERTScore-R': np.float64(0.6338),
 'BERTScore-F1': np.float64(0.6063),
 'Synonym-Aware-ROUGE(SynROUGE)': np.float64(0.0447)}

In [43]:
print_results(predictions)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# mBART

In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MBartForConditionalGeneration, MBart50TokenizerFast

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [23]:
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [24]:
src_lang = "ml_IN"
tgt_lang = "ml_IN"

tokenizer.src_lang = src_lang
tokenizer.tgt_lang = tgt_lang


In [ ]:
def generate_headlines(texts, max_length=64, num_beams=4):
    model.eval()
    preds = []

    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        generated_ids = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True
        )

        pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        preds.append(pred)

    return preds


In [26]:
predicted_headlines = generate_headlines(texts)

In [27]:
predicted_headlines = [re.sub(r"(?i)^\s*\w+:\s*|\s*\([^)]*\)|:\s*", "", p) for p in predicted_headlines]

In [28]:
compute_metrics(predicted_headlines, references)

2025-07-01:13:37:55,212 INFO     [rouge_scorer.py:83] Using default tokenizer.


{'ROUGE-1': np.float64(0.0014),
 'ROUGE-2': np.float64(0.0003),
 'ROUGE-L': np.float64(0.0014),
 'BLEU': 0.0036,
 'BERTScore-P': np.float64(0.6077),
 'BERTScore-R': np.float64(0.5122),
 'BERTScore-F1': np.float64(0.5543),
 'Synonym-Aware-ROUGE(SynROUGE)': np.float64(0.0052)}

In [29]:
print_results(predicted_headlines)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# llama

In [30]:
import torch
from transformers import pipeline

# Load model
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda:1",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:1


In [31]:
# Generate predictions
llama_preds = []

for text in tqdm(texts, desc="Generating Malayalam headlines"):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that generates malayalam news headlines.",
        },
        {
            "role": "user",
            "content": f"""
Generate a clear and concise news headline in Malayalam only based on the following text.

Text (Malayalam): {text}

Important:
- The output must be only a headline in Malayalam.
- Do not use any other language or script.
- Do not include any extra commentary or formatting.
- Do not copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
""",
        },
    ]

    try:
        output = pipe(messages, max_new_tokens=512)[0]["generated_text"]
        llama_preds.append(output)
        torch.cuda.empty_cache()
    except torch.cuda.OutOfMemoryError:
        print("⚠️ CUDA OOM – Skipping this input.")
        llama_preds.append("[OOM ERROR – Skipped]")
        torch.cuda.empty_cache()

Generating Malayalam headlines:   0%|          | 0/500 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_to

KeyboardInterrupt: 

In [ ]:

assistant_contents = [ message['content'] for conversation in llama_preds for message in conversation if message['role'] == 'assistant']


In [ ]:
import re

llama_preds = [re.sub(r"(?i)^\s*Headline:\s*", "", p) for p in assistant_contents]

In [ ]:
compute_metrics(llama_preds, references)

{'ROUGE-1': np.float64(0.0103),
 'ROUGE-2': np.float64(0.0),
 'ROUGE-L': np.float64(0.0103),
 'BLEU': 0.021,
 'BERTScore-P': np.float64(0.7322),
 'BERTScore-R': np.float64(0.7106),
 'BERTScore-F1': np.float64(0.7202)}

In [ ]:
print_results(llama_preds)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

: 

# Bloom

In [8]:
import torch

# Device setup
device = torch.device("cuda:1")
torch.cuda.empty_cache()

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-3b", torch_dtype=torch.float16, device_map="cuda:1").eval()

In [10]:
bloom_preds = []
torch.cuda.empty_cache()

for text in tqdm(texts, desc="Generating Malayalam headlines"):
    try:
        prompt = prompt = f"മലയാളത്തിൽ ഈ വാർത്തക്ക് തലക്കെട്ട് തയ്യാറാക്കുക:\n{text}\nതലക്കെട്ട്:"


        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.inference_mode():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id,
            )

        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        bloom_preds.append(decoded)

        torch.cuda.empty_cache()

    except torch.cuda.OutOfMemoryError:
        print("⚠️ CUDA OOM – Skipping this input.")
        torch.cuda.empty_cache()
        bloom_preds.append("[OOM ERROR – Skipped]")

Generating Malayalam headlines:   0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
import re

bloom_preds = [re.sub(r"(?i)^\s*Headline:\s*", "", p) for p in bloom_preds]

In [12]:
compute_metrics(bloom_preds,references)

2025-07-01:14:03:37,303 INFO     [rouge_scorer.py:83] Using default tokenizer.


{'ROUGE-1': np.float64(0.0331),
 'ROUGE-2': np.float64(0.0065),
 'ROUGE-L': np.float64(0.0328),
 'BLEU': 0.0088,
 'BERTScore-P': np.float64(0.5961),
 'BERTScore-R': np.float64(0.7671),
 'BERTScore-F1': np.float64(0.6705),
 'Synonym-Aware-ROUGE(SynROUGE)': np.float64(0.1263)}

In [ ]:
print_results(bloom_preds)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

: 

# Deepseek 7b

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from datasets import load_dataset
from tqdm import tqdm

# Load DeepSeek Chat model
model_name = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0"
)
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Generate headlines
deepseek_preds = []

batch_size = 4


def create_prompt(text):
    return [
        {"role": "system", "content": "You are a helpful assistant that generates news headlines."},
        {"role": "user", "content": f"""
Generate a clear and concise news headline in Malayalam only based on the following text.

Text (Malayalam): {text}

Important:
- The output must be only a headline in Malayalam.
- Do not use any other language or script.
- Do not include any extra commentary or formatting.
- Do not copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""}
    ]


# for text in tqdm(texts, desc="Generating Malayalam headlines"):
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant that generates news headlines."},
#         {"role": "user", "content": f"""
# Generate a clear and concise news headline in Malayalam only based on the following text.

# Text (Malayalam): {text}

# Important:
# - The output must be only a headline in Malayalam.
# - Do not use any other language or script.
# - Do not include any extra commentary or formatting.
# - Do not copy the text word-for-word.
# - Start your output with: Headline:

# Example (do not include this in your output):
# Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
# """}
#     ]


max_total_tokens = 4096
max_new_tokens = 512

# Calculate max input length allowed
max_input_tokens = max_total_tokens - max_new_tokens


for i in tqdm(range(0, len(texts), batch_size), desc="Generating Malayalam headlines"):
    batch_texts = texts[i:i+batch_size]
    prompts = [create_prompt(text) for text in batch_texts]

    try:
        input_tensor = tokenizer.apply_chat_template(
            prompts,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_input_tokens
        ).to(model.device)

        attention_mask = input_tensor != tokenizer.pad_token_id

        with torch.no_grad():
            outputs = model.generate(input_tensor, attention_mask=attention_mask, max_new_tokens=512)
        
        
        for j in range(len(batch_texts)):
            output_text = tokenizer.decode(
                outputs[j][input_tensor.shape[1]:],
                skip_special_tokens=True
            )
            deepseek_preds.append(output_text)

        # result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
        # deepseek_preds.append(result)
        torch.cuda.empty_cache()

    except torch.cuda.OutOfMemoryError:
        print("⚠️ CUDA OOM – Skipping this input.")
        deepseek_preds.append("[OOM ERROR – Skipped]")
        torch.cuda.empty_cache()


Generating Malayalam headlines:   0%|          | 0/125 [00:00<?, ?it/s]

Generating Malayalam headlines: 100%|██████████| 125/125 [43:34<00:00, 20.92s/it]


In [9]:
import re

deepseek_preds = [re.sub(r"(?i)^\s*Headline:\s*", "", p) for p in deepseek_preds]

In [10]:
compute_metrics(deepseek_preds, references)

{'ROUGE-1': np.float64(0.0236),
 'ROUGE-2': np.float64(0.0077),
 'ROUGE-L': np.float64(0.0239),
 'BLEU': 0.0114,
 'BERTScore-P': np.float64(0.6701),
 'BERTScore-R': np.float64(0.6772),
 'BERTScore-F1': np.float64(0.6727)}

In [11]:
print_results(deepseek_preds)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള